### Quickstart: Compare runs, choose a model and deploy it to a REST API

In [40]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [41]:
df = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [42]:
## Prepare the data
train,test = train_test_split(df,test_size=0.25,random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [ ]:
train_X = train.drop(["quality"],axis=1).values
train_y = train[["quality"]].values.ravel()

## test dataset
test_X = test.drop(["quality"],axis=1).values
test_y = test[["quality"]].values.ravel()

## Spliting into training and validation set
train_X,valid_X,train_y,valid_y = train_test_split(train_X,train_y,test_size=0.2,random_state=42)
singature = infer_signature(train_X,train_y)

In [45]:
print("train_X shape:", train_X.shape)
print("train_y shape:", train_y.shape)
print("valid_X shape:", valid_X.shape)
print("valid_y shape:", valid_y.shape)

train_X shape: (2938, 11)
train_y shape: (2938,)
valid_X shape: (735, 11)
valid_y shape: (735,)


In [44]:
## ANN Model

def train_model(params,epochs,train_X,train_y,valid_X,valid_y,test_X,test_y):

    ## define model architecture
    mean = np.mean(train_X,axis=0)
    var = np.var(train_X,axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_X.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)
        ]
    )

    ## Compile the Model
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=params["lr"],momentum=params["momentum"]),
                  loss="mean_squared_error",
                  metrics=[keras.metrics.RootMeanSquaredError()]
                )
    ## Train the model
    with mlflow.start_run(nested=True):

        model.fit(train_X,train_y,validation_data=(valid_X,valid_y),
                            epochs=epochs,
                            batch_size=64)
        ## Evaluate the model
        eval_result = model.evaluate(valid_X,valid_y,batch_size=64)

        eval_rsme = eval_result[1]

        ## Log the paramter and mertic
        mlflow.log_params(params)
        mlflow.log_metric("eval_rsme",eval_rsme)

        ## Log the model
        mlflow.tensorflow.log_model(model, "model", signature=singature)

        return {"loss": eval_rsme, "status": STATUS_OK, "model": model}


In [35]:
def objective(params):
    ## MLFLOW will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_X = train_X,
        train_y = train_y,
        valid_X = valid_X,
        valid_y = valid_y,
        test_X = test_X,
        test_y = test_y
    )
    return result

In [36]:
space = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [37]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    ## Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo = tpe.suggest,
        max_evals = 4,
        trials = trials
    )

    ## Fetch the details of best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    ## Log the best parameter and mertic
    mlflow.log_params(best)
    mlflow.log_metric("eval_rsme",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=singature)

    ## Print the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rsme: {best_run["loss"]}")

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 33s 749ms/step - loss: 34.9354 - root_mean_squared_error: 5.9106
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.6507 - root_mean_squared_error: 2.7734    
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.6077 - root_mean_squared_error: 2.1928 - val_loss: 0.6452 - val_root_mean_squared_error: 0.8033

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 0.7011 - root_mean_squared_error: 0.8373
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6042 - root_mean_squared_error: 0.7771 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6116 - root_mean_squared_error: 0.7819 - val_loss: 0.5609 - val_root_mean_squared_error: 0.7489

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.6784 - root_mean_squared_error: 0.8237
28/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5827 - root_mean_squared_error: 0.7631 
46/

In [ ]:
## Inferencing
from mlflow.models import validate_serving_input

model_uri = 'runs:/5441692190fb4d46be7e49661de0d42f/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_X)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


array([[3.4740434],
       [8.366061 ],
       [5.431457 ],
       ...,
       [6.71586  ],
       [6.7477694],
       [4.3767366]], dtype=float32)